# classification-11

## What's new:

1- https://chatgpt.com/c/690db8bc-b820-8330-ba21-c6c793d573f9

## next step:

1- Improve labeling ( 3333 from 28-1111 )


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Input, Reshape, TimeDistributed, Lambda, RepeatVector, Dropout, BatchNormalization
from tensorflow.keras import Input, layers, models, callbacks, metrics
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, callbacks
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from sklearn.model_selection import train_test_split



In [ ]:
# 1- Load and Scaling Features

df = pd.read_csv('XAGUSD-197001010000--H1-rates.csv', sep='\t')
# Rename columns for easier access
df.rename(columns={
    '<DATE>': 'DATE',
    '<TIME>': 'TIME',
    '<OPEN>': 'OPEN',
    '<HIGH>': 'HIGH',
    '<LOW>': 'LOW',
    '<CLOSE>': 'CLOSE',
    '<TICKVOL>': 'TICKVOL',
    '<VOL>': 'VOL',
    '<SPREAD>': 'SPREAD'
}, inplace=True)

# ensure strings and strip any weird whitespace
df['DATE'] = df['DATE'].astype(str).str.strip()
df['TIME'] = df['TIME'].astype(str).str.strip()

df['DATETIME'] = pd.to_datetime(df['DATE'] + ' ' + df['TIME'], dayfirst=False, errors='coerce')
if df['DATETIME'].isna().any():
    raise ValueError("Some DATETIME values could not be parsed. Check date/time formats.")

# set DATETIME as index for reindexing
df = df.set_index('DATETIME').sort_index()

# --------------------------
# Create continuous hourly index & fill weekend gaps
# --------------------------
full_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq='h')

# Reindex to full hourly range so weekends/missing hours appear as NaN rows
df = df.reindex(full_index)

# Fill strategy:
# - Prices: forward-fill last known price across weekend gap (common approach for modeling continuity).
# - TICKVOL / VOL: set missing to 0 (no ticks during weekend).
# - SPREAD: forward-fill last known.
# Alternative: you could leave NaNs and drop sequences that cross weekends (safer but reduces data).
df[['OPEN', 'HIGH', 'LOW', 'CLOSE']] = df[['OPEN', 'HIGH', 'LOW', 'CLOSE']].ffill()
df['SPREAD'] = df['SPREAD'].ffill()
df['TICKVOL'] = df['TICKVOL'].fillna(0)
df['VOL'] = df['VOL'].fillna(0)

# Reset index to make DATETIME a regular column again
df = df.reset_index().rename(columns={'index': 'DATETIME'})

In [ ]:
df.shape

In [ ]:
# Example: choose the start and end rows
start_row = 32200
end_row = 33000

# Select the range and make a copy to avoid SettingWithCopyWarning
subset = df.iloc[start_row:end_row + 1].copy()

# Ensure DATETIME is datetime type
subset['DATETIME'] = pd.to_datetime(subset['DATETIME'])

# Plot CLOSE price over time
plt.figure(figsize=(12, 6))
plt.plot(subset['DATETIME'], subset['CLOSE'], linewidth=1.0, color='blue')

# Labels and formatting
plt.title(f"Price Chart from Row {start_row} to {end_row}", fontsize=14)
plt.xlabel("Datetime", fontsize=12)
plt.ylabel("Close Price", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


In [ ]:
# Specify how many rows to remove for model
nn = 33000  # Delete the first nn rows that do not follow the one-hour timeframe.
mm = 500  # Remove mm last row that the model should not see.

# Delete first nn and last mm rows
df_model = df.iloc[nn:len(df) - mm].reset_index(drop=True)

In [ ]:
def label_reversal_points(prices, window=6, threshold=0.0007):
    """
    Labels trend reversals (1=Buy, 2=Sell) based on local mean shifts.
    Smaller window & threshold increase sensitivity.
    """
    prices = np.asarray(prices)
    labels = [0] * len(prices)
    prev_trend = 0  # 1 = up, -1 = down, 0 = unknown

    for i in range(len(prices) - window):
        past = prices[i:i + window // 2]
        future = prices[i + window // 2:i + window]

        past_mean = np.mean(past)
        future_mean = np.mean(future)
        change = (future_mean - past_mean) / past_mean

        if change > threshold:
            curr_trend = 1  # Uptrend
        elif change < -threshold:
            curr_trend = -1  # Downtrend
        else:
            curr_trend = 0  # No significant trend

        # Detect reversal only when trend flips clearly
        if prev_trend == -1 and curr_trend == 1:
            labels[i + window // 2] = 1  # Buy
        elif prev_trend == 1 and curr_trend == -1:
            labels[i + window // 2] = 2  # Sell

        if curr_trend != 0:
            prev_trend = curr_trend

    return labels


df_model['Label'] = label_reversal_points(df_model['CLOSE'].values)

In [ ]:
print(df_model['Label'].value_counts().sort_index())  # 0, 1, 2

In [ ]:
def plot_labeled_candles(df, n=190):
    """
    Plots the last n candles with BUY/SELL labels based on the 'Label' column.
    Assumes df already has a 'DATETIME' column.
    """
    # Drop NaN rows (e.g., weekend gaps)
    df_plot = df.dropna(subset=['CLOSE']).tail(n).copy()

    # Ensure DATETIME is a datetime column (optional safeguard)
    if not pd.api.types.is_datetime64_any_dtype(df_plot['DATETIME']):
        df_plot['DATETIME'] = pd.to_datetime(df_plot['DATETIME'])

    # === Plot Close Price ===
    plt.figure(figsize=(15, 6))
    plt.plot(df_plot['DATETIME'], df_plot['CLOSE'], label='Close Price', color='black', linewidth=1.5)

    # === Plot BUY (1) and SELL (2) signals ===
    for _, row in df_plot.iterrows():
        if row['Label'] == 1:  # BUY
            plt.axvline(x=row['DATETIME'], color='green', linestyle='--', linewidth=1)
            plt.text(row['DATETIME'], row['CLOSE'], 'BUY', color='green', ha='center', va='bottom', fontsize=9)
        elif row['Label'] == 2:  # SELL
            plt.axvline(x=row['DATETIME'], color='red', linestyle='--', linewidth=1)
            plt.text(row['DATETIME'], row['CLOSE'], 'SELL', color='red', ha='center', va='top', fontsize=9)

    # === Aesthetics ===
    plt.title(f'Last {n} Candles with Trend Reversal Labels')
    plt.xlabel('Datetime')
    plt.ylabel('Close Price')
    plt.xticks(rotation=45)
    plt.grid(True, linestyle='--', alpha=0.4)
    plt.tight_layout()
    plt.legend()
    plt.show()



In [ ]:
plot_labeled_candles(df_model)

In [ ]:
# === Forex time-series classification: train + predict ===
# Assumptions:
# - df_model: pandas.DataFrame with continuous hourly rows (~130k) and columns:
#   ['DATETIME', 'DATE','TIME','OPEN','HIGH','LOW','CLOSE','TICKVOL','VOL','SPREAD','Label']
# - df: another DataFrame with same structure (used for prediction input)
# - Both have a DATETIME column parseable by pd.to_datetime
# - FEATURES to use: ['OPEN','HIGH','LOW','CLOSE','TICKVOL']
#
# Final output (last expression) is `predicted_df` with 10 rows:
#   DATETIME, forecast_class, prob_0, prob_1, prob_2

# --------------------------
# User hyperparameters
# --------------------------
WINDOW_SIZE = 60
FORECAST_HORIZON = 10
FEATURES = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'TICKVOL']
N_CLASSES = 3  # 0=no signal,1=buy,2=sell

# Training hyperparams (tweak as needed)
VALIDATION_SPLIT = 0.1
MODEL_SAVE_PATH = "best_model_seq2seq.keras"

# --------------------------
# 0) Basic checks & prepare datetimes
# --------------------------
# Ensure DATETIME column is datetime dtype
for dfi in (df_model, df):
    if not np.issubdtype(dfi['DATETIME'].dtype, np.datetime64):
        dfi['DATETIME'] = pd.to_datetime(dfi['DATETIME'], infer_datetime_format=True, dayfirst=False)

# Ensure feature columns exist
missing = [c for c in FEATURES if c not in df_model.columns]
if missing:
    raise ValueError(f"Missing feature columns in df_model: {missing}")

# --------------------------
# 1) Create sliding windows X and targets y from df_model
#    For each time t (index i) where i >= WINDOW_SIZE-1 and i+FORECAST_HORIZON < len(df_model),
#    X sample uses rows [i-WINDOW_SIZE+1 ... i] (WINDOW_SIZE rows)
#    Targets are Label at i+1 ... i+FORECAST_HORIZON (FORECAST_HORIZON labels)
# --------------------------
labels = df_model['Label'].astype(int).values
data_values = df_model[FEATURES].values  # shape (T, n_features)
T = len(df_model)

X_list = []
y_list = []

# We can form samples for i in [WINDOW_SIZE-1, T-FORECAST_HORIZON-1]
start_idx = WINDOW_SIZE - 1
end_idx = T - FORECAST_HORIZON - 1

for i in range(start_idx, end_idx + 1):
    x_window = data_values[i - WINDOW_SIZE + 1 : i + 1]  # shape (WINDOW_SIZE, n_features)
    y_horizon = labels[i + 1 : i + 1 + FORECAST_HORIZON]  # shape (FORECAST_HORIZON,)
    # sanity: ensure length
    if x_window.shape[0] != WINDOW_SIZE or y_horizon.shape[0] != FORECAST_HORIZON:
        continue
    X_list.append(x_window)
    y_list.append(y_horizon)

X = np.stack(X_list, axis=0)  # shape (N, WINDOW_SIZE, n_features)
y = np.stack(y_list, axis=0)  # shape (N, FORECAST_HORIZON)

print("Prepared dataset shapes:", X.shape, y.shape)

# --------------------------
# 2) Train / validation split
# --------------------------
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=VALIDATION_SPLIT, shuffle=True, random_state=42)

# --------------------------
# 3) Scaling: fit scaler on training X (flatten time axis)
# --------------------------
# We'll fit a StandardScaler on the feature columns across all timesteps in the training set
scaler = StandardScaler()
# reshape to (N*WINDOW_SIZE, n_features)
X_train_2d = X_train.reshape(-1, X_train.shape[2])
scaler.fit(X_train_2d)

# transform both train and val
def scale_X(X_in):
    n_samples, win, n_feat = X_in.shape
    flat = X_in.reshape(-1, n_feat)
    flat_scaled = scaler.transform(flat)
    return flat_scaled.reshape(n_samples, win, n_feat)

X_train_scaled = scale_X(X_train)
X_val_scaled = scale_X(X_val)

# --------------------------
# 4) Convert targets to one-hot (for categorical crossentropy)
# --------------------------
y_train_cat = tf.keras.utils.to_categorical(y_train, num_classes=N_CLASSES)  # shape (N, H, C)
y_val_cat = tf.keras.utils.to_categorical(y_val, num_classes=N_CLASSES)

# --------------------------
# 5) Handle class imbalance: compute class weights from flattened labels across training set
#    We'll compute weights per class and then create a sample_weight matrix of shape (N, H)
# --------------------------
y_train_flat = y_train.ravel()
class_counts = np.bincount(y_train_flat, minlength=N_CLASSES)
total = y_train_flat.shape[0]
# Avoid divide-by-zero
class_freq = np.maximum(class_counts, 1)
class_weights = total / (class_freq * N_CLASSES)  # inverse-frequency normalized
print("Class counts (train):", class_counts)
print("Computed class_weights:", class_weights)

# sample_weight matrix shape (N_samples, FORECAST_HORIZON)
# map each label to its class weight
sample_weight_train = np.vectorize(lambda lbl: class_weights[lbl])(y_train)
sample_weight_val = np.vectorize(lambda lbl: class_weights[lbl])(y_val)

# --------------------------
# 6) Build Seq2Seq-style model (encoder -> RepeatVector -> decoder LSTM with return_sequences)
#    Output: TimeDistributed Dense softmax with FORECAST_HORIZON time steps
# --------------------------
n_features = X_train_scaled.shape[2]
latent_units = 128
drop_rate = 0.2

inp = Input(shape=(WINDOW_SIZE, n_features), name="encoder_input")
x = LSTM(latent_units, return_sequences=False, name="encoder_lstm")(inp)
x = Dropout(drop_rate)(x)
x = BatchNormalization()(x)
# Repeat
x = RepeatVector(FORECAST_HORIZON)(x)
# Decoder
x = LSTM(128, return_sequences=True, name="decoder_lstm")(x)
x = Dropout(drop_rate)(x)
x = TimeDistributed(Dense(64, activation='relu'))(x)
out = TimeDistributed(Dense(N_CLASSES, activation='softmax'), name='out')(x)

model = Model(inp, out)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


In [ ]:
# --------------------------
# 7) Training callbacks
# --------------------------
es = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True, verbose=1)
mc = ModelCheckpoint(MODEL_SAVE_PATH, monitor='val_loss', save_best_only=True, verbose=1)

# Note: Keras accepts sample_weight shaped (samples, timesteps) for temporal weighting
# We'll provide sample_weight_train and sample_weight_val which are (N, FORECAST_HORIZON)
history = model.fit(
    X_train_scaled, y_train_cat,
    validation_data=(X_val_scaled, y_val_cat, sample_weight_val),
    epochs=100,
    batch_size=256,
    callbacks=[es, mc],
    sample_weight=sample_weight_train,
    verbose=1
)

# Save final scaler + model if desired
model.save("final_model.keras")  # recommended native Keras format

In [ ]:
# --------------------------
# PREDICTION SECTION
# --------------------------
# Specification:
# - Use df (not df_model) to select 60 unseen candles ending at given_time
# - given_time = "2025.08.13 21:00:00" (user-provided format)
# - prepare the input, scale it, run model.predict, get probs and classes, build predicted_df
# --------------------------

given_time = "2025.08.13 21:00:00"
given_time_dt = pd.to_datetime(given_time, infer_datetime_format=True)

# find the index in df where DATETIME == given_time_dt
if not np.issubdtype(df['DATETIME'].dtype, np.datetime64):
    df['DATETIME'] = pd.to_datetime(df['DATETIME'], infer_datetime_format=True)

matches = df.index[df['DATETIME'] == given_time_dt].tolist()
if len(matches) == 0:
    raise ValueError(f"given_time {given_time_dt} not found exactly in df['DATETIME']. If index is different, adjust or ensure times match.")
idx = matches[0]

# Ensure we have at least WINDOW_SIZE rows ending at idx (inclusive)
if idx - (WINDOW_SIZE - 1) < 0:
    raise ValueError(f"Not enough history before given_time index {idx} to build a window of size {WINDOW_SIZE}.")

input_window_df = df.iloc[idx - (WINDOW_SIZE - 1) : idx + 1].copy()  # inclusive end
# sanity check
assert len(input_window_df) == WINDOW_SIZE

# select features and scale
X_input = input_window_df[FEATURES].values.reshape(1, WINDOW_SIZE, n_features)  # shape (1,60,n_feat)
# check feature count
if X_input.shape[2] != n_features:
    raise ValueError(f"Feature dimension mismatch: model expects {n_features} features, but X_input has {X_input.shape[2]}")

# scale with previously fitted scaler
X_input_flat = X_input.reshape(-1, n_features)
X_input_scaled_flat = scaler.transform(X_input_flat)
X_input_scaled = X_input_scaled_flat.reshape(1, WINDOW_SIZE, n_features)

# Run prediction
y_pred_prob = model.predict(X_input_scaled)  # shape (1, FORECAST_HORIZON, N_CLASSES)
y_pred_prob = np.squeeze(y_pred_prob, axis=0)  # (FORECAST_HORIZON, N_CLASSES)
y_pred_class = np.argmax(y_pred_prob, axis=1)  # (FORECAST_HORIZON,)

# Build forecast DATETIME list: next 10 hours immediately after given_time
forecast_datetimes = [given_time_dt + pd.Timedelta(hours=i+1) for i in range(FORECAST_HORIZON)]

predicted_df = pd.DataFrame({
    'DATETIME': forecast_datetimes,
    'forecast_class': y_pred_class,
    'prob_0': y_pred_prob[:, 0],
    'prob_1': y_pred_prob[:, 1],
    'prob_2': y_pred_prob[:, 2],
})

# Final display/return: predicted_df is the last expression (as required)
predicted_df

# plot section

In [ ]:
# --------------------------
# === Visualization Block ===
# --------------------------

# --- 1. Historical window (last 4 real candles before forecast) ---
# --- Find the starting index ---
start_idx = df.index[df['DATETIME'] == pd.to_datetime(given_time)][0]

# --- 1. Extract next n candles ---
input_df = df.iloc[start_idx: start_idx + WINDOW_SIZE].copy()

historical_df = input_df.tail(4).copy()
historical_df

In [ ]:
# --- 2. Actual future 10 candles  ---
# Since input_df ends at index (start_idx - 1), actual_future_df starts right after that.
actual_future_start = start_idx
actual_future_end = start_idx + FORECAST_HORIZON
actual_future_df = df.iloc[actual_future_start - 1:actual_future_end].copy()
actual_future_df

In [ ]:
# --- 3. Create predicted_df (forecast for next 10 hours) ---
last_timestamp = input_df['DATETIME'].iloc[-1]
datetime_index = pd.date_range(
    start=last_timestamp + pd.Timedelta(hours=1),
    periods=FORECAST_HORIZON,
    freq='h'
)


# --- 4. Add text labels for clarity ---
predicted_df['label'] = predicted_df['forecast_class'].map({1: 'buy', 2: 'sell'}).fillna('')


# --- 5. Plot title & output settings ---
plot_title = 'Actual vs Predicted Forex Trend Reversals'
output_plot_path = None  # e.g., 'forecast_plot.png'



In [ ]:
# --- 6. Import your plotting utility ---

import sys
sys.path.insert(1, '../utils')
import forex_plot_utils_2

# --- 7. Plot all series ---
forex_plot_utils_2.plot_all_series(
    historical_df=historical_df,
    predicted_df=predicted_df,
    actual_future_df=actual_future_df,
    title=plot_title,
    output_path=output_plot_path
)


In [ ]:
# 11- Save Model

from datetime import datetime
import os

# 11-1 Create timestamp and paths
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
model_filename = f'model_{timestamp}.keras'
model_path = os.path.join('saved_models', model_filename)

# 11-2 Directory to hold logs and extras
log_dir = os.path.join('saved_models', f'model_{timestamp}_logs')
os.makedirs(log_dir, exist_ok=True)

# 11-3 Save model
model.save(model_path)

# 11-4 Save training history
history_df = pd.DataFrame(history.history)
history_df.to_csv(os.path.join(log_dir, 'training_history.csv'), index=False)

# 11-5 Save training loss plot
plt.figure()
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(log_dir, 'training_loss.png'))
plt.close()

# 4. Evaluate on validation set (since no X_test/y_test defined)
final_train_loss = history.history['loss'][-1]
final_train_acc = history.history['cat_acc'][-1]
final_val_loss, final_val_acc = model.evaluate(X_val_scaled, y_val_cat, verbose=0)

# 5. Save model summary and performance metrics
summary_path = os.path.join(log_dir, 'model_log.txt')
with open(summary_path, 'w') as f:
    model.summary(print_fn=lambda x: f.write(x + '\n'))
    f.write('\n')
    f.write(f'Final Training Loss: {final_train_loss:.6f}\n')
    f.write(f'Final Training Accuracy: {final_train_acc:.6f}\n')
    f.write(f'Final Validation Loss: {final_val_loss:.6f}\n')
    f.write(f'Final Validation Accuracy: {final_val_acc:.6f}\n')

In [ ]:
model_path = 'saved_models/model_20251106_214146.keras'
model = keras.models.load_model(
    model_path,
    custom_objects={'loss_fn': focal_loss(), 'focal_loss': focal_loss()},
    safe_mode=False
)